In [1]:
import pandas as pd
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc
# import dash_core_components as dcc 
# import dash_html_components as html
from dash.dependencies import Input, Output, State
import datetime

# from visualize import Visualize2
import random
# from google.cloud import bigquery
# from google.oauth2 import service_account
import pandas as pd
import numpy as np
import json

In [2]:
# credentials = service_account.Credentials.from_service_account_file(
# 'chain-coders-5d47d58b2c33.json')

# project_id = 'chain-coders'
# client = bigquery.Client(credentials= credentials,project=project_id)
# query = "SELECT * FROM `redfin.redfin-data-state`"
# df = client.query(query).to_dataframe()

In [3]:
app = dash.Dash(__name__)

In [4]:
# #get us states json file
# file = "us-states.json"
# with open("{}".format(file)) as user_file:
#     geojson = json.load(user_file)

In [5]:
# -- Import and clean data (importing csv into pandas)
df = pd.read_csv("state_market_tracker.csv")

#format data type for time
df['period_begin'] = pd.to_datetime(df['period_begin'], format="%m/%d/%Y").dt.date
df['period_end'] = pd.to_datetime(df['period_end'], format="%m/%d/%Y").dt.date

# #format data type for time
# df['period_begin'] = pd.to_datetime(df['period_begin'], format="%Y/%m/%d").dt.date
# df['period_end'] = pd.to_datetime(df['period_end'], format="%Y/%m/%d").dt.date

#property types for drop down
property_set = list(set(df['property_type'].values.tolist()))

#metric types for dropdown
metric_set = ['median_sale_price', 'median_ppsf', 'median_sale_price_mom', 'median_ppsf_mom']

#dates for slider
sorted_dates = np.sort(df['period_end'].unique())
sorted_dates = ['{}'.format(i) for i in sorted_dates] #convert each date to string within a list
counter = 0
year = 2012
slider_labels = {}
#labels for slider
for index, date in enumerate(sorted_dates):
       if counter == index:
              slider_labels['{}'.format(index)] = year
              year += 1
              counter += 12
       else: 
              slider_labels['{}'.format(index)] = ''

# labels = {index: date for index, date in enumerate(sorted_dates)} #labels of dates for slider object

#color-scale
color_scale = [[0.0, '#edf8fb'],[0.2, '#ccece6'],[0.4, '#99d8c9'],
       [0.6, '#66c2a4'],[0.8, '#2ca25f'],[1.0, '#006d2c']] # purples


In [6]:
print(slider_labels)


{'0': 2012, '1': '', '2': '', '3': '', '4': '', '5': '', '6': '', '7': '', '8': '', '9': '', '10': '', '11': '', '12': 2013, '13': '', '14': '', '15': '', '16': '', '17': '', '18': '', '19': '', '20': '', '21': '', '22': '', '23': '', '24': 2014, '25': '', '26': '', '27': '', '28': '', '29': '', '30': '', '31': '', '32': '', '33': '', '34': '', '35': '', '36': 2015, '37': '', '38': '', '39': '', '40': '', '41': '', '42': '', '43': '', '44': '', '45': '', '46': '', '47': '', '48': 2016, '49': '', '50': '', '51': '', '52': '', '53': '', '54': '', '55': '', '56': '', '57': '', '58': '', '59': '', '60': 2017, '61': '', '62': '', '63': '', '64': '', '65': '', '66': '', '67': '', '68': '', '69': '', '70': '', '71': '', '72': 2018, '73': '', '74': '', '75': '', '76': '', '77': '', '78': '', '79': '', '80': '', '81': '', '82': '', '83': '', '84': 2019, '85': '', '86': '', '87': '', '88': '', '89': '', '90': '', '91': '', '92': '', '93': '', '94': '', '95': '', '96': 2020, '97': '', '98': '', '

In [7]:
df.columns

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [8]:
print(property_set)

['Condo/Co-op', 'All Residential', 'Single Family Residential', 'Townhouse', 'Multi-Family (2-4 Unit)']


In [9]:
print(sorted_dates)
print(len(sorted_dates))

['2012-01-31', '2012-02-29', '2012-03-31', '2012-04-30', '2012-05-31', '2012-06-30', '2012-07-31', '2012-08-31', '2012-09-30', '2012-10-31', '2012-11-30', '2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017

In [10]:
# line = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2])])

In [11]:
df2 = pd.read_csv("dummy_data.csv")
df2.head(10)


,year,state,median_ppsf,median_sale_price,property
0,2023,MN,483,274053,Condo/Co-op
1,2024,MN,38,271021,Condo/Co-op
2,2025,MN,700,419935,Condo/Co-op
3,2026,MN,27,482962,Condo/Co-op
4,2023,MN,685,223464,All Residential
5,2024,MN,667,360849,All Residential
6,2025,MN,552,716118,All Residential
7,2026,MN,93,210531,All Residential
8,2023,MN,526,387056,Townhouse
9,2024,MN,280,402558,Townhouse


In [12]:
# temp = df2[(df2['property'] == 'All Residential') & (df2['state'] == 'MN')]
# print(temp)

In [13]:
# ------------------------------------------------------------------------------
# App layout
#what goes inside dash layout are the dash components and any html we need
app.layout = html.Div([

    html.H1("Redfin Choropleth Data", style={'text-align': 'center'}), #html Title

    html.Div(id='output_container', children=["Metric"], style = {'font-weight': 'bold'}),

    #components, e.g., drop downs, sliders
    #first drop down by metric
    dcc.Dropdown(id="metric",
                 placeholder = "Select Metric",
                 options=metric_set,
                 value=metric_set[0],
                 style={'width': "60%",}
                 ),

    html.Br(), #space between dropdowns

    html.Div(id='output_container', style = {'font-weight': 'bold'}, 
    children = ["Property Type"]
    ),

    #second drop down for property type
    dcc.Dropdown(id="property_type",
                 placeholder = "Select Property Type",
                 options=property_set,
                 value=property_set[0],
                 style={'width': "60%"},
                 ),

    
    html.Br(), #space
    
    #create div element, e.g., text to display dropdown selection
    html.Div(id='output_container', children=[]),

    html.Br(), #space

    #graph object, e.g., choropleth
    dcc.Graph(id='choropleth', figure={}),

    

    #slider object
    dcc.Slider(0, len(sorted_dates), step = None, #min value of 0 and max of number of unique dates
               value=0, #where the slider starts
               marks = slider_labels,
               tooltip={"placement": "bottom", "always_visible": True}, #place slider at bottom of graph
               id='date_chosen'),

    #add line chart
    dcc.Graph(id = "line_chart", figure = {})

])

In [14]:
# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    Output(component_id='output_container', component_property='children'),
    Output(component_id='choropleth', component_property='figure'),
    Output(component_id = 'line_chart', component_property = 'figure'),
    Input(component_id='property_type', component_property='value'),
    Input(component_id = 'metric', component_property = 'value'),
    Input(component_id='date_chosen', component_property='value'),
    Input(component_id = 'choropleth', component_property = 'hoverData'))


def update_graph(property_type, metric, date_chosen, hover):
    print(property_type)
    print(type(property_type))
    # print(hover)
    # print(type(hover))

    container = "Date Selected: {}".format(sorted_dates[date_chosen])

    dff = df.copy()
    dff = dff[['period_begin', 'period_end', 'state_code', 'median_sale_price', 'median_sale_price_mom', 
    'median_ppsf', 'median_ppsf_mom', 'property_type']]
    dff = dff[dff['property_type'] == property_type]
    dff = dff[dff['period_end'] == pd.to_datetime(sorted_dates[date_chosen], format="%Y/%m/%d")]


    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='{}'.format(metric),
        hover_data=['state_code', '{}'.format(metric)],
        color_continuous_scale= color_scale
    )

    if type(hover) == dict:
        hover_state = hover['points'][0]['customdata'][0]
        hover_df = df2[(df2['property'] == property_type) & (df2['state'] == hover_state)]
        # hover_df = hover_df[['year', '{}'.format(metric)]]
        # hover_x = hover_df['year'].values.tolist()
        # hover_y = hover_df['{}'.format(metric)].values.tolist()
        # hover_y = [int(year) for year in hover_y]
        line_data = px.line(hover_df, x = "year", y = '{}'.format(metric), title = 'Predicted {fmetric} for {fstate}'.format(fmetric = metric, fstate = hover_state))
        # line_data = go.Figure(data=[go.Scatter(x=hover_x, y=hover_y)])
        result = line_data
    else:
        result = []

    #try using px.line
    

    # if type(hover) == dict:
    #     hover_state = hover['points'][0]['customdata'][0]
    #     hover_df = df2[(df2['property'] == property_type) & (df2['state'] == hover_state)]
    #     hover_df = hover_df[['year', '{}'.format(metric)]]
    #     hover_x = hover_df['year'].values.tolist()
    #     hover_y = hover_df['{}'.format(metric)].values.tolist()
    #     line_data = go.Figure(data=[go.Scatter(x=hover_x, y=hover_y)])
    #     print(hover_state)
    #     print(line_data)
    #     if hover_state == "MN":
    #         result = line_data
    # else:
    #     result = []

    # # Plotly Express using geojson data
    # fig = px.choropleth(
    #     dff,
    #     geojson = geojson,
    #     # featureidkey="id",
    #     locations='state_code',
    #     scope="usa",
    #     color='{}'.format(metric),
    #     hover_data=['state_code', '{}'.format(metric)],
    #     color_continuous_scale= color_scale
    # )




    return container, fig, result

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)